In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import json
import pandas as pd

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


In [6]:
options = Options()
options.headless = False 
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
options.add_argument("disable-infobars")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [7]:

import requests

url = "https://www.zara.com/de/de/woman-must-have-l4108.html?v1=2420954"

In [8]:
driver.get(url)
time.sleep(5)

In [9]:
wait = WebDriverWait(driver, 10)

In [10]:
time.sleep(3)

In [11]:
def gradual_scroll(driver, pause_time, step_size):
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        for i in range(0, last_height, step_size):
            driver.execute_script(f"window.scrollTo(0, {i});")
            time.sleep(pause_time)
        
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        last_height = new_height

# Perform gradual scrolling with pauses and small steps
gradual_scroll(driver, pause_time=0.2, step_size=50)

# Additional wait to ensure that all content has been loaded
time.sleep(5)

# Extract the HTML from the page
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# Close the browser
driver.quit()

# Save the HTML to a file for further analysis (commented out)
# with open("zara_woman_blazer.html", "w", encoding='utf-8') as file:
#     file.write(page_source)

print("Scrolling, view change, and extraction have been completed.")

KeyboardInterrupt: 

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from time import sleep


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Navigate to the Zara Woman Germany webpage
driver.get("https://www.zara.com/de/de/damen-neuware-l1180.html?v1=2419517")

# Retry logic in case of StaleElementReferenceException
attempt = 0
max_attempts = 3

while attempt < max_attempts:
    try:
        # Check for iframe and switch to it
        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'iframe'))  # Locate the iframe if present
        )
        driver.switch_to.frame(iframe)
        print("Switched to iframe.")

        # Locate the cookie consent button inside the iframe
        cookie_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'ALLE COOKIE')]"))
        )

        # Add a brief wait to ensure stability before clicking (optional)
        sleep(1)

        # Click the cookie consent button
        cookie_button.click()
        print("Cookie consent accepted.")

        # Switch back to the default content after interacting with the iframe
        driver.switch_to.default_content()

        # Exit the loop as the task was successful
        break

    except StaleElementReferenceException:
        print(f"Stale element, retrying... attempt {attempt + 1}")
        attempt += 1
        driver.switch_to.default_content()  # Ensure we return to the default content before retrying

    except TimeoutException:
        print("No cookie consent banner found or timeout occurred.")
        break

if attempt == max_attempts:
    print(f"Failed after {max_attempts} attempts.")

# Now proceed with your scraping logic or further interactions
try:
    # Example of waiting for the product items to load (adjust the class name)
    WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'correct-product-item-class'))
    )
    print("Products loaded successfully.")
except TimeoutException:
    print("Timed out waiting for the products to load.")

# Close the driver
driver.quit()

Stale element, retrying... attempt 1
Switched to iframe.
No cookie consent banner found or timeout occurred.
Timed out waiting for the products to load.


In [34]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

In [35]:
last_height = driver.execute_script("return document.documentElement.scrollHeight")

In [37]:
products = soup.find_all('li', class_='product-grid-product')

In [ ]:
products

In [39]:
data = {
    "Categoria":[],
    "Sexo": [], 
    "Producto": [],
    "Color": [],
    "Subcategoría": [],
    "Referencia": [],
    "Precio": [],
    "Imagen": [],
    "Link": []
}

In [ ]:
for product in products:
    # Extraer el nombre y color del producto desde el atributo 'alt' de <img>
    image_tag = product.find('img', class_='media-image__image')
    if image_tag and 'alt' in image_tag.attrs:
        alt_text = image_tag['alt']
        # Verificar si se puede dividir en dos partes: nombre y color
        split_text = alt_text.split(' - ')
        if len(split_text) == 2:
            name, color = split_text
        else:
            name = alt_text  # Si no hay un guion, tomamos todo como el nombre
            color = "No Color"  # Color por defecto
    else:
        name = "No Name"
        color = "No Color"
    
    # Extraer el enlace al producto desde el <a> con clase 'product-link'
    link = product.find('a', class_='product-link')['href'] if product.find('a', class_='product-link') else "No Link"
    
    # Extraer la subcategoría desde el link, basándonos en palabras clave
    subcategory = link.split('/')[-1].split('-')[0]  # Primera palabra de la URL del producto (ej. "capa")

    # Extraer el precio del producto desde el <span> con clase 'money-amount__main'
    price = product.find('span', class_='money-amount__main').get_text(strip=True) if product.find('span', class_='money-amount__main') else "No Price"
    
    # Extraer la referencia del producto (viene como 'data-productid' en la etiqueta <li>)
    reference = product.get('data-productid', 'No Reference')
    
    # Extraer la imagen del producto desde el <img> con clase 'media-image__image'
    image = product.find('img', class_='media-image__image')['src'] if product.find('img', class_='media-image__image') else "No Image"
    
    categoria = categoria

    sexo = sexo 

    # Guardar la información en el diccionario
    data["Categoria"].append(categoria)
    data["Sexo"].append(sexo)
    data["Producto"].append(name)
    data["Color"].append(color)
    data["Subcategoría"].append(subcategory)
    data["Referencia"].append(reference)
    data["Precio"].append(price)
    data["Imagen"].append(image)
    data["Link"].append(link)

# Convertir el diccionario en un DataFrame de pandas
df = pd.DataFrame(data)